In [1]:
import pandas as pd
import glob
import os
import csv
import numpy as np
from scipy.stats import chi2_contingency
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
%matplotlib inline

try:
    code_dir
except NameError:
    code_dir = os.getcwd()
    source_dir = code_dir.replace("codes_local", "0_sources")
    out_dir = code_dir.replace("codes_local", "2_ChIP_peak_jaccard")

## 1. Calculate jaccard index between ChIP associated genes

In [2]:
gene_list_file = source_dir + "/gene_list.csv"
gene_list_df = pd.read_csv(gene_list_file)
gene_list_labels_uniq = set(gene_list_df['labels'])
print(gene_list_labels_uniq)

chip_anno_dir = '/media/pipkin/Rocket2/T_Cell_ChIP/202012_ChIP/3_peak_annotations'
chip_anno_files = glob.glob("%s/*annoDf.csv"%chip_anno_dir)

{'sc_d8_arm_up', 'sc_d5_arm_up', 'sc_P9_arm_up', 'sc_P4_cl13_up', 'sc_P7_cl13_up', 'd5_KLRG1hi_Runx3_promoted', 'nascent_all', 'sc_d8_cl13_up', 'nascent_Il2_promoted', 'd5_KLRG1hi_Runx3_repressed', 'sc_d5_cl13_up', 'sc_P2_arm_up', 'sc_P7_arm_up', 'sc_P4_arm_up', 'sc_P3_cl13_up', 'sc_P2_cl13_up', 'nascent_48h_dn', 'sc_P9_cl13_up', 'sc_P3_arm_up', 'd5_KLRG1lo_Runx3_promoted', 'd5_KLRG1lo_all', 'sc_P6_cl13_up', 'd5_KLRG1lo_Runx3_repressed', 'sc_P5_cl13_up', 'sc_all', 'nascent_Il2_repressed', 'nascent_48h_up', 'd5_KLRG1hi_all', 'sc_P6_arm_up', 'sc_P5_arm_up'}


### 1.1 Calculate jaccard index for RNAseq correlated ChIP associated genes

In [3]:
chip_names = [x.split("/")[-1].replace("_annoDf.csv", "") for x in chip_anno_files]
jaccard_df = pd.DataFrame({"ChIP": chip_names})
chip_genes_n = []
for chip_i_file in chip_anno_files:
    chip_i_df = pd.read_csv(chip_i_file)
    chip_i_genes = set(x for x in chip_i_df['SYMBOL'] if str(x) != 'nan')
    chip_genes_n.append(len(chip_i_genes))
jaccard_df["ChIP gene #"] = chip_genes_n
jaccard_df

,ChIP,ChIP gene #
0,2021_GoldrathLab_Brd4___BRD4_TE-CD8,8410
1,2021_GoldrathLab_Brd4___BRD4_NAV-CD8,7333
2,2021_GoldrathLab_Brd4___BRD4_MP-CD8,7086
3,2020_PNAS_KwesiMaliepaard___H3K79me2_naive-CD8,8091
4,2020_PNAS_KwesiMaliepaard___H3K79me2_mem-CD8,7425
...,...,...
80,2013_PLOS_ONE_Lotem___Runx3_invitro-Il2-CD8,2698
81,2013_PLOS_ONE_Lotem___H3K4me1_invitro-NAV-CD8,15785
82,2013_PLOS_ONE_Lotem___H3K4me1_invitro-Il2-CD8,3264
83,2013_IMMUNITY_KIM___Foxo1_naive-CD8-WT,2178


In [5]:
completed = []

In [ ]:
for gene_list_label_x in gene_list_labels_uniq:   
    if gene_list_label_x not in completed:
        print("-----", gene_list_label_x, "-----")
        jaccard_df_x = jaccard_df.copy()
        gene_list = set(gene_list_df[gene_list_df['labels'] == gene_list_label_x]['gene_names'].tolist())
        for chip_i_file in chip_anno_files:
            chip_i_jaccard = []
            chip_i_name = chip_i_file.split("/")[-1].replace("_annoDf.csv", "")
            chip_i_df = pd.read_csv(chip_i_file)
            chip_i_genes = set(x for x in chip_i_df['SYMBOL'] if str(x) != 'nan')
            for chip_j_file in chip_anno_files:
                chip_j_name = chip_j_file.split("/")[-1].replace("_annoDf.csv", "")
                chip_j_df = pd.read_csv(chip_j_file)
                chip_j_genes = set(x for x in chip_j_df['SYMBOL'] if str(x) != 'nan')
                chip_i_overlap = gene_list & chip_i_genes
                chip_j_overlap = gene_list & chip_j_genes
                if len(chip_i_overlap | chip_j_overlap) != 0:
                    ij_jaccard = len(chip_i_overlap & chip_j_overlap) / len(chip_i_overlap | chip_j_overlap)
                else:
                    ij_jaccard = 0
                chip_i_jaccard.append(ij_jaccard)
            jaccard_df_x[chip_i_name] = ["{:.4f}".format(x) for x in chip_i_jaccard]
        jaccard_df_x.to_csv("%s/%s_jaccard.csv"%(out_dir, gene_list_label_x), index=False)
    completed.append(gene_list_label_x)

----- sc_d8_arm_up -----


### 1.2 Calculate jaccard index for ChIP associated genes

In [ ]:
for chip_i_file in chip_anno_files:
    chip_i_jaccard = []
    chip_i_name = chip_i_file.split("/")[-1].replace("_annoDf.csv", "")
    chip_i_df = pd.read_csv(chip_i_file)
    chip_i_genes = set(x for x in chip_i_df['SYMBOL'] if str(x) != 'nan')
    for chip_j_file in chip_anno_files:
        chip_j_name = chip_j_file.split("/")[-1].replace("_annoDf.csv", "")
        chip_j_df = pd.read_csv(chip_j_file)
        chip_j_genes = set(x for x in chip_j_df['SYMBOL'] if str(x) != 'nan')
        chip_i_overlap = chip_i_genes
        chip_j_overlap = chip_j_genes
        ij_jaccard = len(chip_i_overlap & chip_j_overlap) / len(chip_i_overlap | chip_j_overlap)
        chip_i_jaccard.append(ij_jaccard)
    jaccard_df[chip_i_name] = ["{:.4f}".format(x) for x in chip_i_jaccard]
jaccard_df.to_csv("%s/ChIP_jaccard.csv"%out_dir, index=False)   

## 2. Plot jaccard index heatmap (correlation between ChIPs)

In [ ]:
jaccard_files = glob.glob("%s/*jaccard.csv"%out_dir)
for jaccard_file in jaccard_files:
    jaccard_df = pd.read_csv(jaccard_file)
    jaccard_heatmap_df = jaccard_df[chip_names]
    jaccard_heatmap_df.index = jaccard_df['ChIP']

    jaccard_clustermap = sns.clustermap(jaccard_heatmap_df, figsize=(30,30))
    jaccard_clustermap.savefig(jaccard_file.replace(".csv", "_clustermap.jpg"))

    jaccard_dgram = jaccard_clustermap.dendrogram_col.dendrogram
    jaccard_order = [chip_names[int(x)] for x in jaccard_dgram['ivl']]
    jaccard_order_df = pd.DataFrame({"ChIP": jaccard_order})
    jaccard_order_df.to_csv(jaccard_file.replace(".csv", "_clustermapOrder.csv"))

## 3. Summarize jaccard index between ChIPs for different RNAseq association

In [ ]:
# Use only non redudant combinations for ChIP experiment comparisons
uniq_chip_cbs = list(itertools.combinations(chip_names,2))
uniq_chip_cbs = ["---".join(x) for x in uniq_chip_cbs]

In [ ]:
jaccard_all_df = pd.DataFrame({"ChIP_combination": uniq_chip_cbs})
for jaccard_file in jaccard_files:
    jaccard_name = jaccard_file.split("/")[-1].replace("_jaccard.csv", "")
    jaccard_df = pd.read_csv(jaccard_file)
    
    jaccard_df_melt = pd.melt(jaccard_df, id_vars=['ChIP', 'ChIP gene #'])
    jaccard_df_melt.columns = ['ChIP_1', 'ChIP gene #', 'ChIP_2', 'jaccard_index']
    jaccard_df_melt['ChIP_combination'] = ["---".join([x,y]) for index, (x, y) in 
                                           enumerate(zip(jaccard_df_melt['ChIP_1'].tolist(), 
                                                         jaccard_df_melt['ChIP_2'].tolist()))]
    jaccard_df_melt = jaccard_df_melt[['ChIP_combination', 'jaccard_index']]
    jaccard_df_melt = jaccard_df_melt.set_index("ChIP_combination")
    jaccard_df_melt = jaccard_df_melt.loc[uniq_chip_cbs]
    
    jaccard_all_df[jaccard_name] = jaccard_df_melt['jaccard_index'].tolist()
jaccard_all_df.to_csv("All_jaccard_index.csv", index=False)

In [ ]:
jaccard_all_clustermap_df = jaccard_all_df.set_index("ChIP_combination") 

In [ ]:
jaccard_all_clustermap = sns.clustermap(jaccard_all_clustermap_df, figsize=(30,30))
jaccard_all_clustermap.savefig("%s/All_jaccard_index_clustermap.jpg"%out_dir)
plt.close()

jaccard_all_dgram = jaccard_all_clustermap.dendrogram_col.dendrogram
jaccard_all_order = [jaccard_all_clustermap_df.columns.tolist()[int(x)] for x in jaccard_all_dgram['ivl']]
jaccard_all_order_df = pd.DataFrame({"ChIP": jaccard_all_order})
jaccard_all_order_df.to_csv("%s/All_jaccard_index_clustermapColOrder.csv"%out_dir)

jaccard_all_dgram = jaccard_all_clustermap.dendrogram_row.dendrogram
jaccard_all_order = [jaccard_all_clustermap_df.index.tolist()[int(x)] for x in jaccard_all_dgram['ivl']]
jaccard_all_order_df = pd.DataFrame({"ChIP": jaccard_all_order})
jaccard_all_order_df.to_csv("%s/All_jaccard_index_clustermapRowOrder.csv"%out_dir)